# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('kc_house_data.csv')

Finished parsing file /home/newuser/venv/course2/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.562664 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/newuser/venv/course2/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.40355 secs.

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
l1_penalty = 1e10
l2_penalty = 0
my_output = 'price'

In [6]:
def polynomial_regression(data,output,features,l1_penalty,l2_penalty):

    model = turicreate.linear_regression.create(sales, target='price', features=features,
                                                validation_set=None, 
                                                l2_penalty=l2_penalty, l1_penalty=l1_penalty,
                                                verbose = False)
    return model


Find what features had non-zero weight.

In [7]:
model_all =polynomial_regression(sales,'price',all_features,l1_penalty,l2_penalty)

In [8]:
import numpy as np

In [9]:
def print_coefficients(model):
    deg = len(model.coefficients['value'])-1
    w = list(model.coefficients['value'])
    print('Learned polynomial for degree ' + str(deg) + ':')
    w.reverse()
    print(np.poly1d(w))

In [10]:
print_coefficients(model_all)

Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05


In [11]:
print ('number of nonzeros = %d' % (model_all.coefficients['value']).nnz())

number of nonzeros = 6


In [12]:
model_all.coefficients

name,index,value,stderr
(intercept),None,274873.0556516683,None
bedrooms,None,0.0,None
bedrooms_square,None,0.0,None
bathrooms,None,8468.531052082517,None
sqft_living,None,24.420721103749127,None
sqft_living_sqrt,None,350.0605510046117,None
sqft_lot,None,0.0,None
sqft_lot_sqrt,None,0.0,None
floors,None,0.0,None
floors_square,None,0.0,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [13]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [14]:
rss_number = []
for l1_penalty in np.logspace(1,7,num=13):
    
    model = polynomial_regression(training,'price',all_features,l1_penalty,0.)
    predictions = model.predict(validation)
    errors = predictions - validation['price']
    RSS = (pow(errors,2)).sum()
    rss_number.append(RSS)
    print (l1_penalty,'/',RSS)
    

10.0 / 630973678898876.9
31.622776601683793 / 630973679045988.0
100.0 / 630973679511194.4
316.22776601683796 / 630973680982305.4
1000.0 / 630973685634376.4
3162.2776601683795 / 630973700345587.9
10000.0 / 630973746867257.9
31622.776601683792 / 630973893989020.2
100000.0 / 630974359302061.9
316227.7660168379 / 630975831483230.4
1000000.0 / 630980494248545.1
3162277.6601683795 / 630995312409343.2
10000000.0 / 631042903558233.5


In [15]:
print(min(rss_number))

630973678898876.9


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [16]:
model = polynomial_regression(training,'price',all_features,10.0,0.)
print ('number of nonzeros = %d' % (model.coefficients['value']).nnz())

number of nonzeros = 18




***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [17]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [18]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [19]:
rss_number = []
for l1_penalty in l1_penalty_values:
    
    model = polynomial_regression(training,'price',all_features,l1_penalty,0.)
    predictions = model.predict(validation)
    errors = predictions - validation['price']
    RSS = (pow(errors,2)).sum()
    rss_number.append(RSS)
    print ('number of nonzeros = %d' % (model.coefficients['value']).nnz())
    print (l1_penalty,'/',RSS)
    

number of nonzeros = 18
100000000.0 / 631772981140101.8
number of nonzeros = 18
127427498.57031322 / 632067022373050.6
number of nonzeros = 18
162377673.91887242 / 632451760061999.1
number of nonzeros = 18
206913808.111479 / 632960113735221.9
number of nonzeros = 18
263665089.87303555 / 633672115886020.6
number of nonzeros = 18
335981828.6283788 / 634690663237861.5
number of nonzeros = 18
428133239.8719396 / 636283608127973.4
number of nonzeros = 17
545559478.1168514 / 638537511924832.0
number of nonzeros = 17
695192796.1775591 / 641861901688966.2
number of nonzeros = 17
885866790.4100832 / 646760503717577.9
number of nonzeros = 17
1128837891.6846883 / 654246389201976.4
number of nonzeros = 17
1438449888.2876658 / 665740014393148.2
number of nonzeros = 16
1832980710.8324375 / 682579276013997.0
number of nonzeros = 16
2335721469.0901213 / 706393768559719.4
number of nonzeros = 16
2976351441.6313133 / 743862938143165.9
number of nonzeros = 15
3792690190.7322536 / 798960997622890.9
number

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [20]:
l1_penalty_min_rss = min(rss_number)
l1_penalty_max_rss = max(rss_number)
print('The smallest l1_penalty rss = ',l1_penalty_min_rss)
print('The largest l1_penalty rss = ',l1_penalty_max_rss)

The smallest l1_penalty rss =  631772981140101.8
The largest l1_penalty rss =  1205879980028371.2


In [21]:
smallest_l1_penalty = min(l1_penalty_values)
largest_l1_penalty = max(l1_penalty_values)
print('The smallest l1_penalty  = ',smallest_l1_penalty)
print('The largest l1_penalty  = ',largest_l1_penalty)

The smallest l1_penalty  =  100000000.0
The largest l1_penalty  =  10000000000.0


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [22]:
l1_penalty_values = np.linspace(smallest_l1_penalty,largest_l1_penalty,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [23]:
rss_number = []

for l1_penalty in l1_penalty_values:    
    model = polynomial_regression(training,'price',all_features,l1_penalty,0.)
    predictions = model.predict(validation)
    errors = predictions - validation['price']
    RSS = (pow(errors,2)).sum()
    rss_number.append(RSS)
    print ('number of nonzeros = %d' % (model.coefficients['value']).nnz())
    print (l1_penalty,'/',RSS)
        
    

number of nonzeros = 18
100000000.0 / 631772981140101.8
number of nonzeros = 17
621052631.5789473 / 640112093034714.1
number of nonzeros = 17
1142105263.1578946 / 654691237472126.9
number of nonzeros = 17
1663157894.7368422 / 675475198220296.0
number of nonzeros = 16
2184210526.315789 / 698716234040111.5
number of nonzeros = 16
2705263157.894737 / 727011006569465.2
number of nonzeros = 16
3226315789.4736843 / 760684305549021.4
number of nonzeros = 15
3747368421.0526314 / 796076777906257.2
number of nonzeros = 13
4268421052.631579 / 825975321717767.6
number of nonzeros = 13
4789473684.210526 / 856090123511988.9
number of nonzeros = 12
5310526315.789474 / 897913419682375.8
number of nonzeros = 11
5831578947.368421 / 939878682517340.4
number of nonzeros = 10
6352631578.947369 / 975085699043191.2
number of nonzeros = 10
6873684210.526316 / 1013298819264039.2
number of nonzeros = 8
7394736842.105263 / 1047597705449170.8
number of nonzeros = 7
7915789473.684211 / 1077559279966657.9
number of

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [24]:
model = polynomial_regression(training,'price',all_features,l1_penalty=7915789473.684211,l2_penalty=0.)
print_coefficients(model)

Learned polynomial for degree 17:
       14        13       5         4             3
27.63 x  + 2787 x  + 659 x + 30.48 x + 1.483e+04 x + 616.4 x + 2.278e+05


In [25]:
for i in model.coefficients['name','value']:    
        print(i)

{'name': '(intercept)', 'value': 227796.51558423977}
{'name': 'bedrooms', 'value': 616.4077016179333}
{'name': 'bedrooms_square', 'value': 0.0}
{'name': 'bathrooms', 'value': 14826.16828351347}
{'name': 'sqft_living', 'value': 30.483329746722088}
{'name': 'sqft_living_sqrt', 'value': 659.0089988270271}
{'name': 'sqft_lot', 'value': 0.0}
{'name': 'sqft_lot_sqrt', 'value': 0.0}
{'name': 'floors', 'value': 0.0}
{'name': 'floors_square', 'value': 0.0}
{'name': 'waterfront', 'value': 0.0}
{'name': 'view', 'value': 0.0}
{'name': 'condition', 'value': 0.0}
{'name': 'grade', 'value': 2787.2516998587844}
{'name': 'sqft_above', 'value': 27.629750103138903}
{'name': 'sqft_basement', 'value': 0.0}
{'name': 'yr_built', 'value': 0.0}
{'name': 'yr_renovated', 'value': 0.0}


In [26]:
prediction = model.predict(testing)
prediction[0]

424705.48525652074

In [27]:
testing[0]

{'id': 3793500160,
 'date': '20150312T000000',
 'price': 323000.0,
 'bedrooms': 3,
 'bathrooms': 2.5,
 'sqft_living': 1890,
 'sqft_lot': 6560,
 'floors': 2.0,
 'waterfront': 0,
 'view': 0,
 'condition': 3,
 'grade': 7,
 'sqft_above': 1890,
 'sqft_basement': 0,
 'yr_built': 2003,
 'yr_renovated': 0,
 'zipcode': 98038,
 'lat': 47.3684,
 'long': -122.031,
 'sqft_living15': 2390,
 'sqft_lot15': 7570,
 'sqft_living_sqrt': 43.474130238568314,
 'sqft_lot_sqrt': 80.99382692526635,
 'bedrooms_square': 9,
 'floors_square': 4.0}